In [1]:
import pandas as pd
from sklearn.dummy import DummyClassifier

extractedDirectory = 'extractedData'

rawDataDir = '/'.join([extractedDirectory,'hirise-map-proj-v3_2'])

In [2]:
imageMap = pd.read_csv(
    '/'.join([rawDataDir,"labels-map-proj_v3_2_train_val_test.txt"]), 
    header = None,
    names = ['imageName','class','imageSet'],
    delimiter = ' '    
)
imageMap

,imageName,class,imageSet
0,ESP_013049_0950_RED-0067.jpg,7,train
1,ESP_013049_0950_RED-0067-fv.jpg,7,train
2,ESP_013049_0950_RED-0067-brt.jpg,7,train
3,ESP_013049_0950_RED-0067-r90.jpg,7,train
4,ESP_013049_0950_RED-0067-r180.jpg,7,train
...,...,...,...
67805,ESP_018707_2205_RED-0041.jpg,0,test
67806,ESP_018707_2205_RED-0062.jpg,0,test
67807,ESP_018707_2205_RED-0105.jpg,0,test
67808,ESP_018707_2205_RED-0058.jpg,0,test


In [3]:
classDecoder = pd.read_csv(
    '/'.join([rawDataDir,'landmarks_map-proj-v3_2_classmap.csv']),
    header= None,
    names = ['class','className']
)
classDecoder

,class,className
0,0,other
1,1,crater
2,2,dark dune
3,3,slope streak
4,4,bright dune
5,5,impact ejecta
6,6,swiss cheese
7,7,spider


In [4]:
imageMap = imageMap.merge(classDecoder,on='class').drop_duplicates()
imageMap

,imageName,class,imageSet,className
0,ESP_013049_0950_RED-0067.jpg,7,train,spider
1,ESP_013049_0950_RED-0067-fv.jpg,7,train,spider
2,ESP_013049_0950_RED-0067-brt.jpg,7,train,spider
3,ESP_013049_0950_RED-0067-r90.jpg,7,train,spider
4,ESP_013049_0950_RED-0067-r180.jpg,7,train,spider
...,...,...,...,...
67805,ESP_016613_2570_RED-0038.jpg,4,test,bright dune
67806,ESP_016613_2570_RED-0006.jpg,4,test,bright dune
67807,ESP_016613_2570_RED-0043.jpg,4,test,bright dune
67808,ESP_016613_2570_RED-0044.jpg,4,test,bright dune


In [5]:
imageMap.className.value_counts()

other            52722
crater            5024
swiss cheese      1834
bright dune       1654
slope streak      1575
spider             896
dark dune          766
impact ejecta      476
Name: className, dtype: int64

The dataset is quite heavily imbalanced. 81% of the dataset is 'other' with 'crater' taking up another sizeable of chunk of the rest. Lets have a quick look at the breakdown by each train/val/test set

In [6]:
imageMap[imageMap.imageSet =='train'].className.value_counts(normalize=True)

other            0.817064
crater           0.073460
bright dune      0.031871
swiss cheese     0.026154
slope streak     0.018865
spider           0.013006
dark dune        0.012577
impact ejecta    0.007003
Name: className, dtype: float64

In [7]:
imageMap[imageMap.imageSet =='val'].className.value_counts(normalize=True)

other            0.791605
crater           0.094321
slope streak     0.042469
swiss cheese     0.036049
spider           0.015309
impact ejecta    0.008889
dark dune        0.005926
bright dune      0.005432
Name: className, dtype: float64

In [8]:
imageMap[imageMap.imageSet =='test'].className.value_counts(normalize=True)

other            0.826548
crater           0.049637
dark dune        0.036810
slope streak     0.027328
spider           0.023424
swiss cheese     0.023424
bright dune      0.008924
impact ejecta    0.003904
Name: className, dtype: float64

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Normalizing data for 8 bit
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'images/train/',
    target_size = (227,227),
    batch_size=225,
    color_mode='grayscale',
    class_mode='categorical'
)

Found 48979 images belonging to 8 classes.


In [13]:
dumdum = DummyClassifier(strategy='most_frequent')

In [20]:
dumdum.fit(train_generator.filenames,train_generator.classes)
dumdum.score(train_generator.filenames,train_generator.classes)

0.8170644561955124